In [3]:
!pip install "tensorflow>=1.7.0"

!pip install tensorflow-hub

    100% |████████████████████████████████| 48.0MB 27kB/s 
    100% |████████████████████████████████| 3.1MB 439kB/s 
  Found existing installation: tensorboard 1.6.0
    Uninstalling tensorboard-1.6.0:
      Successfully uninstalled tensorboard-1.6.0
  Found existing installation: tensorflow 1.6.0
    Uninstalling tensorflow-1.6.0:
      Successfully uninstalled tensorflow-1.6.0
    100% |████████████████████████████████| 61kB 3.3MB/s 


In [0]:
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'1.7.0'

## prepare the Iris data

In [0]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [0]:
def maybe_download():
  train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
  test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)
  
  return train_path, test_path

In [0]:
def load_data(y_name='Species'):
  train_path, test_path = maybe_download()
  
  train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
  train_x, train_y = train, train.pop(y_name)
  
  test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
  test_x, test_y = test, test.pop(y_name)
  
  return (train_x, train_y), (test_x, test_y)

In [0]:
def train_input_fn(features, labels, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  
  dataset = dataset.shuffle(1000).repeat().batch(batch_size)
  
  return dataset

In [0]:
def eval_input_fn(features, labels, batch_size):
  features = dict(features)
  
  if labels is None:
    inputs = features
  else:
    inputs = (features, labels)
    
  dataset = tf.data.Dataset.from_tensor_slices(inputs)
  
  # Batch the examples
  assert batch_size is not None, "batch_size must not be None"
  dataset = dataset.batch(batch_size)
  
  return dataset

## Estimator

In [0]:
# fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

In [23]:
train_x.shape

(120, 4)

In [26]:
# Feature columns describe how to use the input
my_feature_columns = []
for key in train_x.keys():
  print(key)
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

SepalLength
SepalWidth
PetalLength
PetalWidth


In [27]:
my_feature_columns

[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [25]:
# Build 2 hidden layer DNN with 10, 10 units respectively
classifier = tf.estimator.DNNClassifier(
  feature_columns=my_feature_columns,
  hidden_units=[10, 10],
  n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpjwxbnbgp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f824243ca58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
batch_size=32
train_steps=100

In [41]:
# Trin the Model
classifier.train(
  input_fn=lambda:train_input_fn(train_x, train_y,
                                 batch_size),
  steps=train_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjwxbnbgp/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 201 into /tmp/tmpjwxbnbgp/model.ckpt.
INFO:tensorflow:loss = 1.7632289, step = 201
INFO:tensorflow:Saving checkpoints for 300 into /tmp/tmpjwxbnbgp/model.ckpt.
INFO:tensorflow:Loss for final step: 2.4586205.


In [44]:
# Evaluate the model
eval_result = classifier.evaluate(
  input_fn=lambda:eval_input_fn(test_x, test_y,
                                batch_size))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-16-06:31:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjwxbnbgp/model.ckpt-300
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-16-06:31:50
INFO:tensorflow:Saving dict for global step 300: accuracy = 0.96666664, average_loss = 0.09250528, global_step = 300, loss = 2.7751584


In [45]:
eval_result

{'accuracy': 0.96666664,
 'average_loss': 0.09250528,
 'global_step': 300,
 'loss': 2.7751584}

In [46]:
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))



Test set accuracy: 0.967



In [0]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

In [0]:
predictions = classifier.predict(
  input_fn=lambda:eval_input_fn(predict_x,
                                labels=None,
                                batch_size=batch_size))

In [59]:
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(predictions, expected):
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]
  
  print(template.format(SPECIES[class_id],
                        100 * probability, expec))
  

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjwxbnbgp/model.ckpt-300
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (99.6%), expected "Setosa"

Prediction is "Versicolor" (97.2%), expected "Versicolor"

Prediction is "Virginica" (92.3%), expected "Virginica"
